<a href="https://colab.research.google.com/github/raafitt/GoogleColab/blob/Linear-Regression/Regression_web_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install  schedule nest_asyncio streamlit altair autokeras==1.1.0 tensorflow==2.15.1 keras-nlp==0.5.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of tensorflow-text to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.1/527.1 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 92.5 MB/s eta 0

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler,RobustScaler,MaxAbsScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import mean_squared_error
import pandas as pd
import seaborn as sns
import numpy as np
import autokeras as ak
from sklearn.linear_model import LinearRegression,LogisticRegression
from tensorflow.keras.callbacks import Callback


Вспомогательный файл, возвращающий прогноз

In [ ]:
%%writefile regression_model.py
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import mean_squared_error
import pandas as pd
import seaborn as sns
import numpy as np
import autokeras as ak
from sklearn.linear_model import LinearRegression,LogisticRegression


from tensorflow import keras
model=keras.models.load_model('/content/best_regression_model_inter_val_mse_0_5.keras')



def get_pred(x,model,i,j):
  pred=model.predict(np.reshape(np.reshape(x,(74, 352))[i],(1,352)))
  return pred[0][j]

if __name__ == "__get_pred__":
    get_pred()

Writing regression_model.py


# Файл с приложением

In [ ]:
%%writefile main.py
import streamlit as st
import numpy as np
import io
import pandas as pd
from collections import deque
import re
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from io import StringIO
import altair as alt
import aiohttp
import asyncio
import json
import nest_asyncio
from regression_model import get_pred
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import mean_squared_error
import pandas as pd
import seaborn as sns
import numpy as np
import autokeras as ak
from sklearn.linear_model import LinearRegression,LogisticRegression
from tensorflow import keras

# Загрузка модели и данных
pd.options.display.float_format = '{:.2f}'.format

model=keras.models.load_model('/content/best_regression_model_inter_val_mse_0_5.keras')
x_df = pd.read_csv('/content/x_test.csv').drop('Unnamed: 0', axis=1)
y_df = pd.read_csv('/content/y_test.csv').drop('Unnamed: 0', axis=1)
x = np.array(x_df).astype('float32')
y = np.array(y_df).astype('float32')




features_dict=[]
for i in x_df.columns:
  features_dict.append({'name':i,'position':x_df.columns.get_loc(i)})
features_dict=features_dict[:32]
# Создание списка имен компонентов для выбора
features = ["Выберите входной параметр для отображения"] + [feature["name"] for feature in features_dict]
# Интерфейс выбора
selected_feature = st.selectbox("", features)
if selected_feature != "Выберите входной параметр для отображения":
    # Найти словарь, соответствующий выбранному компоненту
    selected_data = next((feature for feature in features_dict if feature["name"] == selected_feature), None)


# Выбор компонента для отображения
comp_pos = None
component = st.selectbox("Выберите компонент для отображения:", ("", "ИПБ", "Бензол", "ДИПБ"))
if component == 'ИПБ':
    comp_pos = 3
elif component == 'Бензол':
    comp_pos = 7
elif component == 'ДИПБ':
    comp_pos = 8


if st.button('Запустить прогноз состава РМА'):
    t_data = deque(maxlen=8)  # Очередь для первого графика
    data = deque(maxlen=7)  # Очередь для второго графика
    t_chart = st.empty()
    product_chart = st.empty()

    async def update_first_chart(k,selected_feature):
        """Обновление первого графика."""
        pos=selected_feature['position']
        feature_name=selected_feature['name']
        feature = np.round_(x[k:k + 8, pos],2)
        # Создаём DataFrame с исходными значениями
        t_df = pd.DataFrame({'index': np.linspace(0, len(data) - 1, 71), f'{feature_name}': np.nan})

        # Размещаем исходные значения на соответствующих индексах
        t_df.loc[::10, f'{feature_name}'] = feature

        # Интерполируем пустоты (линейная интерполяция)
        t_df[f'{feature_name}'] = t_df[f'{feature_name}'].interpolate(method='linear')
        t_df[f'{feature_name}'] = t_df[f'{feature_name}'].apply(lambda x: round(x, 3) if pd.notnull(x) else x)


        # Устанавливаем временные метки (опционально)
        t_df['time'] = pd.date_range(start=pd.Timestamp.now().replace(second=0, microsecond=0), periods=len(t_df), freq='6T')

        y_min_T = t_df[f'{feature_name}'].min()-t_df[f'{feature_name}'].min()*0.0001
        y_max_T = t_df[f'{feature_name}'].max()+t_df[f'{feature_name}'].max()*0.0001

        for i in range(1,71):
          df=t_df.iloc[0:i,1:3]
          # Построение первого графика
          # Линия для TICA_103
          feature_line = alt.Chart(df).mark_line(color='red').encode(
              x=alt.X('time:T', axis=alt.Axis(format='%M', title='Минуты')),
              y=alt.Y(f'{feature_name}:Q', scale=alt.Scale(domain=[y_min_T, y_max_T]), title='Значения')
          )

          df_subset = df.iloc[::7]

          # Точки на линии TICA_103
          feature_points = alt.Chart(df_subset).mark_point(color='red').encode(
              x='time:T',
              y=f'{feature_name}:Q'
          )



          # Текст над точкамиTICA_103
          feature_text = alt.Chart(df_subset).mark_text(align='left', dx=5, dy=-5, color='red').encode(
              x='time:T',
              y=f'{feature_name}:Q',
              text=f'{feature_name}:Q'
          )


          # Объединение всех частей в один график
          final_t_chart = (
              feature_line + feature_points + feature_text
          ).properties(
              title=f'График изменения {feature_name}',
              width=666,
              height=360
          )

          t_chart.altair_chart(final_t_chart)
          await asyncio.sleep(0.05)

    async def update_second_chart(i):
        """Обновление второго графика."""
        pred_value = get_pred(x,model,i,comp_pos)
        real_value = y[i, comp_pos]
        data.append({'pred_value': pred_value, 'real_value': real_value, 'real_type':'Истиное значение','pred_type':'Предсказанное значение'})
        df = pd.DataFrame(data)
        df['time'] = pd.date_range(start=pd.Timestamp.now().replace(second=0, microsecond=0), periods=len(df), freq='1h')
        df['pred_value'] = df['pred_value'].apply(lambda x: round(x, 3) if pd.notnull(x) else x)
        df['real_value'] = df['real_value'].apply(lambda x: round(x, 3) if pd.notnull(x) else x)
        y_min_pred = df['pred_value'].min()-df['pred_value'].min()*0.0001
        y_max_pred = df['pred_value'].max()+df['pred_value'].max()*0.0001

        # Линия для предсказанных значений
        pred_value_line = alt.Chart(df).mark_line().encode(
            x=alt.X('time:T', axis=alt.Axis(format='%H', title='Часы')),
            y=alt.Y('pred_value:Q', scale=alt.Scale(domain=[y_min_pred, y_max_pred]), title='Значения'),
            color=alt.Color('pred_type:N', legend=alt.Legend(title="Тип значения",orient='bottom'))  # Легенда по типу
        )

        # Линия для реальных значений
        real_value_line = alt.Chart(df).mark_line(color='blue').encode(
            x='time:T',
            y='real_value:Q',
            color=alt.Color('real_type:N', legend=alt.Legend(title="Тип значения",orient='bottom'))  # Легенда по типу
        )

        # Точки на линии предсказанных значений
        pred_value_points = alt.Chart(df).mark_point().encode(
            x='time:T',
            y='pred_value:Q'
        )

        # Точки на линии реальных значений
        real_value_points = alt.Chart(df).mark_point(color='blue').encode(
            x='time:T',
            y='real_value:Q'
        )

        # Текст над точками предсказанных значений
        pred_value_text = alt.Chart(df).mark_text(align='left', dx=5, dy=-10).encode(
            x='time:T',
            y='pred_value:Q',
            text='pred_value:Q'
        )

        # Текст над точками реальных значений
        real_value_text = alt.Chart(df).mark_text(align='left', dx=-20, dy=10, color='blue').encode(
            x='time:T',
            y='real_value:Q',
            text='real_value:Q'
        )


        # Объединение всех частей в один график
        final_product_chart = (
            pred_value_line + real_value_line +
            pred_value_points + real_value_points +
            pred_value_text + real_value_text
        ).properties(
            title=f'График изменения содержания {component}',
            width=666,
            height=380
        )
        product_chart.altair_chart(final_product_chart)
        await asyncio.sleep(0.5)

    async def main_loop():
        """Главный цикл обновления графиков."""



        k = 0
        for i in range(50):  # Лимит на 50 итераций
            await update_first_chart(k,selected_data)
            await update_second_chart(i)
            k += 8

    asyncio.run(main_loop())

Overwriting main.py


In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 352)]             0         
                                                                 
 multi_category_encoding (M  (None, 352)               0         
 ultiCategoryEncoding)                                           
                                                                 
 normalization (Normalizati  (None, 352)               705       
 on)                                                             
                                                                 
 dense (Dense)               (None, 32)                11296     
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                1056  

In [ ]:
!streamlit run main.py --server.address=localhost >/content/logs.txt & ssh -o "StrictHostKeyChecking no" -R 80:localhost:8501 serveo.net

Forwarding HTTP traffic from https://9fcd0e25a78d3a92a2ed2b70b516a914.serveo.net
HTTP request from 54.39.207.190 to https://9fcd0e25a78d3a92a2ed2b70b516a914.serveo.net/
HTTP request from 54.39.207.190 to https://9fcd0e25a78d3a92a2ed2b70b516a914.serveo.net/static/media/SourceSansPro-Regular.0d69e5ff5e92ac64a0c9.woff2
HTTP request from 54.39.207.190 to https://9fcd0e25a78d3a92a2ed2b70b516a914.serveo.net/static/media/SourceSansPro-SemiBold.abed79cd0df1827e18cf.woff2
HTTP request from 54.39.207.190 to https://9fcd0e25a78d3a92a2ed2b70b516a914.serveo.net/static/media/SourceSansPro-Bold.118dea98980e20a81ced.woff2
HTTP request from 54.39.207.190 to https://9fcd0e25a78d3a92a2ed2b70b516a914.serveo.net/static/css/main.23bdda6f.css
HTTP request from 54.39.207.190 to https://9fcd0e25a78d3a92a2ed2b70b516a914.serveo.net/static/js/main.dbbac55a.js
HTTP request from 54.39.207.190 to https://9fcd0e25a78d3a92a2ed2b70b516a914.serveo.net/_stcore/health
HTTP request from 54.39.207.190 to https://9fcd0e25a78

In [ ]:
!streamlit run main.py --server.address=localhost >/content/logs.txt & ssh -o "StrictHostKeyChecking no" -R 80:localhost:8501 nokey@localhost.run


Welcome to localhost.run!

Follow your favourite reverse tunnel at [https://twitter.com/localhost_run].

To set up and manage custom domains go to https://admin.localhost.run/

More details on custom domains (and how to enable subdomains of your custom
domain) at https://localhost.run/docs/custom-domains

If you get a permission denied error check the faq for how to connect with a key or
create a free tunnel without a key at [http://localhost:3000/docs/faq#generating-an-ssh-key].

To explore using localhost.run visit the documentation site:
https://localhost.run/docs/


** your connection id is 354a15de-3845-4986-8e40-f7f9eb44a5a6, please mention it if you send me a message about an issue. **

authenticated as anonymous user
f4eadb0aecffc8.lhr.life tunneled with tls termination, https://f4eadb0aecffc8.lhr.life
create an account and add your key for a longer lasting domain name. see https://localhost.run/docs/forever-free/ for more information.
Open your tunnel address on your mobile w